<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/A_Journey_into_the_Depths_of_Neural_Networks/blob/master/Entering_the_World_of_Deep_Neural_Networks_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Entering the World of Deep Neural Networks CNN**

## **Libraries**

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

## **GPU**

In [3]:
#0. Check GPU availability and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## **Hyperparameters**

In [4]:
#1. Hyperparameters
NUM_EPOCHS = 1000
BATCH_SIZE = 64
LEARNING_RATE = 0.05

## **CIFAR-10 Data Preparation**

### **Preprocessing**

In [ ]:
# Transformations: Tensor Conversion and Normalization
# Mean and Standard Deviation Values ​​for CIFAR-10 (Standard)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # مقادیر دقیق‌تر
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # ساده‌تر، برای نرمال‌سازی به بازه [-1, 1]
])